In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.optim as optim
import math
import random

if torch.cuda.is_available():
  dev = 'cuda'
  print("Cuda avaiable. Device used to train the model:", dev)
else: 
  dev = 'cpu'
  print("Cuda not avaiable. Device used to train the model:", dev)


# **A harmonic series dataset**

This dataset generates an input signal and an output signal, given a harmonic distribution and a number of cycles we want to compute.

In [ ]:
# Multishape dataset
class HarmonicSeriesDataset(Dataset):
    """ Class to generate an harmonic series dataset (input+output).
    Args:
    harmonics = harmonic distribution of output signal
    shape = 'step' or 'ramp'
    amplitudes = set of max amplitudes of the signals of the dataset
    n_periods = number of periods to compute
    """
    
    def __init__(self, harmonics, shape='step', amplitudes=[0.25, 0.5, 0.75, 1.0], n_periods=1):
        self.harmonics = torch.tensor(harmonics)
        self.shape = shape if shape in ['step', 'ramp'] else 'step'
        self.len = n_periods*2*harmonics.size()[0]
        self.amplitudes = amplitudes
        self.sample_rate = 2*harmonics.size()[0]
        self.sample_time = 1/(2*harmonics.size()[0])
        self.n_freqs = harmonics.size()[0]
        self.phases = torch.zeros(self.n_freqs)
        self.freqs = torch.linspace(start=0, end=self.n_freqs-1, steps=self.n_freqs)
    
    def __len__(self):
        return 128

    def __getitem__(self, idx):
      
      self.phases[:] = 0
      amp = np.random.choice(self.amplitudes)
      d_1 = np.random.randint(0,self.len/2)
      d_2 = np.random.randint(self.len/2,self.len-2)
      check = np.random.choice([True,False])
      on_off = torch.ones(self.len) if(check) else torch.zeros(self.len)
      on_off[d_1:d_2] = 0 if(check) else 1
      
      # Define Input
      #if self.shape == 'random':
      #  input = torch.rand(self.len)*amp if(check) else torch.zeros(self.len)
      #  input[d_1:d_2] = 0 if(check) else torch.rand(d_2-d_1)*amp
      #else:
      #  input = torch.ones(self.len)*amp if(check) else torch.zeros(self.len)
      #  input[d_1:d_2] = 0 if(check) else amp
      
      input = torch.ones(self.len)*amp if(check) else torch.zeros(self.len)
      input[d_1:d_2] = 0 if(check) else amp
      # Define Outpu
      output = torch.zeros(self.len)
        
      for i in range(self.len): 
        
        if self.shape == 'ramp':
          if i == 0:
            input[i] = self.sample_time if on_off[i] else 0
          else:
            input[i] = input[i-1] + self.sample_time if on_off[i] else 0
          
        # Clip input from 0 to amp
        input[i] = input[i].clip(0,amp)  
        
        self.phases += (2*np.pi*self.freqs/self.sample_rate)
        self.phases = self.phases * on_off[i]
        output[i] = torch.sum(self.harmonics*torch.sin(self.phases))*input[i]    
      
      return input, output

# Dataset
Let's generate a dataset and plot its content.

In [ ]:
# Dataset configuration
period_len = 60
n_periods = 4
# Generate a distribution of harmonics
harmonics = torch.zeros(period_len//2)
harmonics[1] = 1
harmonics[2] = 1
harmonics[3] = 1
harmonics[10] = 1
# Normalize distribution
harmonics = harmonics/torch.sum(harmonics)

# Create dataset
batch_size = 64
amplitudes = np.linspace(0,1,period_len)
train_dataset = HarmonicSeriesDataset(harmonics=harmonics, shape='step', amplitudes=amplitudes, n_periods=n_periods)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

# Sample storage to plot
signal_len = len(train_dataloader)
print(period_len*n_periods)
print(signal_len)

# Examples of dataset
train_samples = torch.zeros(batch_size,period_len*n_periods,2)

train_samples[:,:,0], train_samples[:,:,1] = next(iter(train_dataloader))

print("Samples.shape:", train_samples.shape)
for i in range(5):
  plt.plot(train_samples[i,:,0].detach().cpu().numpy())
  plt.plot(train_samples[i,:,1].detach().cpu().numpy())
  plt.title("TRAINING SAMPLES: Input and Output {}".format(i))
  plt.show()



# **Gated Recurrent Unit (GRU)**

In [ ]:
class GRUmodel(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, num_layers=1):
    super(GRUmodel, self).__init__()
    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    # GRU (input)
    self.gru = nn.GRU(input_size, hidden_dim, num_layers=num_layers, batch_first=True)
    # Fully connected (output)
    self.lin = nn.Linear(hidden_dim, output_size)
    
  # Forward method
  def forward(self, seq):

    input = seq  # input.shape = (batch_size, sequence_length ,input_size)
  
    out, hidden = self.gru(input) # out.shape = (sequence_length, hidden_dim) # hidden.shape = (num_layers, hidden_dim)
    out = self.lin(out) # out.shape = (sequence_length,1)
    return out


In [ ]:
def gru_evaluate_on_data(net, eval_input, target, device, plot=False):

  L = target.shape[1] # sequence length
  batch_size = target.shape[0] # batch size
  random = np.random.randint(0,batch_size-2)
  target = target[random,:,:].unsqueeze(0).to(device) # ([N,L,1])
  # seq = torch.ones([batch_size,L,1]).to(device) # ([N,L,1])
  seq = eval_input[random,:,:].unsqueeze(0).to(device)

  with torch.no_grad():
    pred = net(seq)
      # Calculate evaluation loss
  eval_loss = torch.sum(torch.abs(target-pred))

  # Plotting results
  if(plot):
    fig, ax = plt.subplots()
    print("[TEST ON DATA] Eval loss {}".format(eval_loss))
    ax.plot(seq[0,:,:].detach().cpu().numpy())
    ax.plot(target[0,:,:].detach().cpu().numpy())
    ax.plot(pred[0,:,:].detach().cpu().numpy())
    plt.show()

  return eval_loss
  
def gru_train(net, epochs, train_dataloader, device, save_state='best_gru.pt'):
  
  net.train()

  # Define Loss, Optimizer, ecc.
  criterion = nn.L1Loss()
  optimizer = optim.Adam(net.parameters(), lr=0.001)

  best_eval_loss = np.inf

  # target = train_samples[:,:,1].unsqueeze(2).to(device) #([N,L,1])
  # train = train_samples[:,:,1].unsqueeze(2).to(device) #([N,L,1]) 
  # L = target.shape[1] # sequence length
  # n_batches = target.shape[0] # batch size
  epochs_loss = np.zeros(epochs)

  for epoch in range(epochs): 
    optimizer.zero_grad()
    running_loss = 0.0

    for i,sample in enumerate(train_dataloader):
      seq,target = sample
      target = target.unsqueeze(2).to(device)
      seq = seq.unsqueeze(2).to(device)
      prediction = net(seq)
      loss = criterion(prediction, target) # Calculate loss
      loss.backward() # Backpropagation and calculates gradients
      optimizer.step() # Updates the weights
    # Print statistics
    running_loss += loss.item() # To calculate epochs' mean loss
    plot = True if (epoch%(epochs/16)==0) else False
    eval_loss = gru_evaluate_on_data(net, seq, target, device, plot=plot)
    
    epochs_loss[epoch] = eval_loss # For plotting loss during epochs

    if eval_loss < best_eval_loss:
      best_eval_loss = eval_loss
      torch.save(net.state_dict(), save_state)

    if (epoch%(epochs/20) == 0):
        print('Epoch: {}/{} .............'.format(epoch, epochs), end=' ')
        print("Train Loss: {:.4f} - Best Eval {:.4f}".format(running_loss, best_eval_loss))
    
    running_loss = 0.0

  print("---------- FINISHED TRAINING ----------")

  # Print Eval Loss
  x_ = np.linspace(0, epochs-1, epochs)
  y_ = epochs_loss
  yhat_ = signal.savgol_filter(y_, 101, 3) # Savitzky-Golay filter for smoothing

  fig, ax = plt.subplots()
  ax.plot(x_, y_)
  ax.plot(x_, yhat_)
  ax.set_xlabel('Epochs')
  ax.set_ylabel('Eval Loss') 
  # ax.grid(True) 
  plt.show()

# **Training GRUnet**

In [ ]:
def train_and_test_GRU(hidden_size, n_hidden_layers, train_dataloader, target, epochs, device, save_state='best_gru.pt'):
  #train_len = train_data.size()[2]

  net = GRUmodel(1,1,hidden_size, n_hidden_layers)
  net = net.to(device)

  model_parameters = filter(lambda p: p.requires_grad, net.parameters())
  params = sum([np.prod(p.size()) for p in model_parameters])
  print("[INFO] Number of parameters: {}".format(params))
  
  # TRAINING
  gru_train(net, epochs, train_dataloader, device, save_state)

  # TEST
  net.load_state_dict(torch.load(save_state))  
  net.eval()
  input,target = next(iter(train_dataloader))
  input = input.unsqueeze(2)
  target = target.unsqueeze(2)
  gru_evaluate_on_data(net, input, target, device, plot=True)

  return net

In [ ]:
GRUnet = train_and_test_GRU(hidden_size=64, n_hidden_layers=1, train_dataloader=train_dataloader, target=train_dataloader, epochs=5000, device=dev)

# **Testing Trained GRUnet**

## Load Pre-trained Net for Testing [Optional]

In [ ]:
GRUnet = GRUmodel(1,1,64,1)
GRUnet = GRUnet.to(dev)
GRUnet.load_state_dict(torch.load('./nn_saved_states/multi_1402_dot25.pt'))

## Testing with step/square input

In [ ]:
# Testing with a random input+target from dataloader

seq, target = next(iter(train_dataloader))
batch_size = target.shape[0] # batch size
random = np.random.randint(0,batch_size-2)

seq = seq[random,:].unsqueeze(0).unsqueeze(2).to(dev)
target = target[random,:].unsqueeze(0).unsqueeze(2).to(dev)

# Calculate Output
GRUnet.eval()
out = GRUnet(seq)

eval_loss =torch.sum(torch.abs(target-out))
print("[TEST ON STEP] Eval loss {}".format(eval_loss))

plt.plot(seq[0,:,:].detach().cpu().numpy())
plt.plot(target[0,:,:].detach().cpu().numpy())
plt.plot(out[0,:,:].detach().cpu().numpy())
plt.show()

## Testing with ramp input

In [ ]:
# Testing with a new dataset with amplitudes different from training dataset
# amplitudes = [float(input("Choose input amplitude for testing dataset: "))]
amplitudes = [0.2, 0.4, 0.6, 0.8]
test_dataset = HarmonicSeriesDataset(harmonics=harmonics, shape='ramp', n_periods=n_periods, amplitudes=amplitudes)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

for i in range(100):
    seq, target = next(iter(test_dataloader))
    seq = seq[random,:].unsqueeze(0).unsqueeze(2).to(dev)
    target = target[random,:].unsqueeze(0).unsqueeze(2).to(dev)

    # Calculate Output
    out = GRUnet(seq)
    
    # Calculate Eval Loss
    eval_loss =torch.sum(torch.abs(target-out))
    print("[TEST {} with RAMP] Eval loss {}".format(i,eval_loss))
    
    if eval_loss <= 5.0:
        plt.plot(seq[0,:,:].detach().cpu().numpy())
        plt.plot(target[0,:,:].detach().cpu().numpy())
        plt.plot(out[0,:,:].detach().cpu().numpy())
        plt.show()    

    elif i%10==0:
        plt.plot(seq[0,:,:].detach().cpu().numpy())
        plt.plot(target[0,:,:].detach().cpu().numpy())
        plt.plot(out[0,:,:].detach().cpu().numpy())
        plt.show()